In [ ]:
import os
import torch
import torch.nn as nn

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import skimage.transform as skit
from PIL import Image
from torchvision import transforms
from torchvision import models

from torch.utils.data import TensorDataset, DataLoader

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#csv 가져오기
df_aircondition = pd.read_csv('/content/drive/MyDrive/air/airQuality.csv')

csv_data = df_aircondition.pivot(index='MSRDT', columns='MSRSTE_NM', values=['PM10', 'PM25'])
csv_data.columns = [f"{col[1]}{col[0]}" for col in csv_data.columns]

# #histogram
# hist = csv_data['송파구PM10'].hist(bins=50)

#결측치 제거
csv_data = csv_data[(csv_data==0.0).sum(axis=1)<3]
csv_data.head(10)

,광진구PM10,동대문구PM10,성북구PM10,송파구PM10,중랑구PM10,광진구PM25,동대문구PM25,성북구PM25,송파구PM25,중랑구PM25
MSRDT,,,,,,,,,,
202408261100,37.0,42.0,41.0,51.0,37.0,27.0,29.0,28.0,35.0,26.0
202408261200,44.0,46.0,38.0,49.0,39.0,32.0,31.0,29.0,37.0,31.0
202408261300,52.0,47.0,32.0,50.0,44.0,34.0,34.0,28.0,41.0,41.0
202408261400,42.0,0.0,36.0,45.0,41.0,34.0,0.0,26.0,36.0,34.0
202408261500,43.0,0.0,32.0,47.0,0.0,34.0,0.0,25.0,28.0,35.0
202408271100,19.0,21.0,16.0,24.0,16.0,11.0,15.0,12.0,9.0,10.0
202408271200,20.0,22.0,23.0,22.0,0.0,10.0,12.0,10.0,12.0,0.0
202408271300,20.0,23.0,18.0,22.0,12.0,9.0,11.0,11.0,14.0,8.0
202408271400,0.0,20.0,18.0,23.0,18.0,0.0,10.0,9.0,12.0,11.0


In [ ]:
scaler = MinMaxScaler()
feature = csv_data.copy()

scaled_csv = scaler.fit_transform(feature)
csv_data = pd.DataFrame(scaled_csv, columns=feature.columns, index=feature.index)

#인덱스 int->string
csv_data.index = csv_data.index.map(str)
csv_data.head()

,광진구PM10,동대문구PM10,성북구PM10,송파구PM10,중랑구PM10,광진구PM25,동대문구PM25,성북구PM25,송파구PM25,중랑구PM25
MSRDT,,,,,,,,,,
202408261100,0.711538,0.792453,0.694915,1.000000,0.578125,0.794118,0.852941,0.800000,0.853659,0.634146
202408261200,0.846154,0.867925,0.644068,0.960784,0.609375,0.941176,0.911765,0.828571,0.902439,0.756098
202408261300,1.000000,0.886792,0.542373,0.980392,0.687500,1.000000,1.000000,0.800000,1.000000,1.000000
202408261400,0.807692,0.000000,0.610169,0.882353,0.640625,1.000000,0.000000,0.742857,0.878049,0.829268
202408271100,0.365385,0.396226,0.271186,0.470588,0.250000,0.323529,0.441176,0.342857,0.219512,0.243902


In [ ]:
class ImgPreprocess:
    def __init__(self, img_directory):
        self.preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        self.directory = img_directory

    # 이미지 전처리> 크기 조정 및 텐서 변환
    def resize(self, image_files):
        images = []
        for img_file in image_files:
            img_path = os.path.join(self.directory, img_file)
            image = Image.open(img_path).convert("RGB")
            image = self.preprocess(image)
            images.append(image)
        return images

    # 파일명 날짜 및 시간별로 그룹화
    def sort_img(self):
        img_dataset = {}
        for img_file in os.listdir(self.directory):
            if img_file.endswith(".JPG"):
                date_time = img_file[:12]  # 파일명에서 날짜와 시간 추출하기

                if date_time not in img_dataset:
                    img_dataset[date_time] = []

                img_dataset[date_time].append(img_file)
        return img_dataset

    # 전처리
    def preprocess_img(self):
        sorted_images = self.sort_img()

        processed_images = {}
        for date_time, image_files in sorted_images.items():
            processed_images[date_time] = self.resize(image_files)

        return processed_images

    def save_processed_images(self, processed_images, save_path):
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        for date_time, images in processed_images.items():
            save_file = os.path.join(save_path, f"{date_time}_processed.pt")
            torch.save(images, save_file)

    # 저장된 텐서 불러오기
    def load_processed_images(self, save_path):
        processed_images = {}
        for file_name in os.listdir(save_path):
            if file_name.endswith("_processed.pt"):
                date_time = file_name.split('_processed.pt')[0]
                file_path = os.path.join(save_path, file_name)
                images = torch.load(file_path)
                processed_images[date_time] = images
        return processed_images

# 사용
img_directory = '/content/drive/MyDrive/air/air_img_all'
save_path = '/content/drive/MyDrive/air/processed_images'
# save_path = '/content/drive/MyDrive/air/processed_images_0_1'

target_path = ''

In [ ]:
img_preprocessor = ImgPreprocess(img_directory)

# 이미지 저장
# processed_images = img_preprocessor.preprocess_img()
# img_preprocessor.save_processed_images(processed_images, save_path)

#target 이미지 process
target_processed_images = processed_images(target_path)

# 이미지 불러오기
processed_images = img_preprocessor.load_processed_images(save_path)

<ipython-input-28-c7de5608d493>:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  images = torch.load(file_path)


In [ ]:
def match_img_csv(processed_images, csv_data):
    matched_images = []
    matched_csv = []

    for date_time, images in processed_images.items():
        for image in images:
            if date_time in csv_data.index:
                csv_values = csv_data.loc[date_time].values
                matched_csv.append(csv_values)
                matched_images.append(image)
            else:
                pass

    matched_images = torch.stack(matched_images)
    matched_csv = torch.tensor(matched_csv, dtype=torch.float32)

    return matched_images, matched_csv

# source
source_images, s_matched_csv = match_img_csv(processed_images, csv_data)
source_train_images, source_valid_images, source_train_labels, source_valid_labels = train_test_split(source_images, s_matched_csv, test_size=0.2, random_state=42)

source_train_dataset = TensorDataset(source_train_images, source_train_labels)
source_valid_dataset = TensorDataset(source_valid_images, source_valid_labels)

# target
#target_images, target_labels = match_img_csv(target_processed_images, csv_data)
#target_dataset = TensorDataset(target_images, target_labels)
target_images = target_processed_images
target_dataset = TensorDataset(target_images)


source_train_loader = DataLoader(source_train_dataset, batch_size=32, shuffle=True, num_workers=2, drop_last=False)
source_valid_loader = DataLoader(source_valid_dataset, batch_size=32, shuffle=True, num_workers=2, drop_last=False)
target_loader = DataLoader(target_dataset, batch_size=1, shuffle=False)

In [ ]:
print(f"Total images: {len(matched_images)}")
print(f"Total labels: {len(matched_csv)}")

Total images: 1276
Total labels: 1276


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#model
weights = models.ViT_B_16_Weights.DEFAULT
model = models.vit_b_16(weights=weights)

model.to(device)

for param in model.parameters():
    param.requires_grad = False

cuda:0


In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

In [ ]:
output_size = len(csv_data.columns)

model.heads = nn.Sequential(
    nn.Linear(768, 100),
    nn.ReLU(),
    nn.Linear(100, output_size)
)

model.to(device)

for param in model.heads.parameters():
    param.requires_grad =  True

In [ ]:
from torch.autograd import Function

# Gradient Reverse Layer
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [ ]:
class DomainAdaptationModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = nn.Sequential(model.conv_proj, model.encoder)
        self.regressor = model.heads #이렇게 해도 되나?
        self.classifier = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU()
            nn.Linear(128,2)
        )

    def forward(self,x, domain_lable=None): #False/True
        features = self.feature_extractor(x)
        regression_output = self.regressor(features)

        if domain_label is not None:
            reversed_feature = ReverseLayerF.apply(features, alpha=1.0)
            domain_output = self.classifier(reversed_feature)
            return regression_output, domain_output

        return regression_output

In [ ]:
batch_size=32
loss_fn = MAE()
domain_loss_fn = nn.CrossEntropyLoss()
patience=7

model = DomainAdaptationModel()
model.to(device)

learning_rate = 0.00002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def domain_train(source_train_loader, target_loader, model, loss_fn, domain_loss_fn, optimizer, epoch):
    model.train()
    total_reg_loss, total_domain_loss  = 0.0, 0.0
    total_samples = 0

    for (source_data, source_label),(target_data,_) in zip(source_train_loader, target_loader):
        source_data, source_label = source_data.to(device), source_label.to(device)
        target_data = target_data.to(device)

        source_domain_label = torch.zeros(len(source_data), dtype=torch.long).to(device)
        target_domain_label = torch.ones(len(target_data), dtype=torch.long).to(device)
        domain_label = torch.cat([source_domain_label, target_domain_label], dim=0)

        train_data = torch.cat([source_data, target_data], dim=0)
        regression_output, domain_output = model(train_data, domain_label=True)

        reg_loss = loss_fn(regression_output[:len(source_data)], source_label)
        domain_loss = domain_loss_fn(domain_output, domain_label)

        total_loss = reg_loss + domain_loss

        optimizer.zero_grad() #학습 도중에 optimizer 0으로 설정
        total_loss.backward()
        optimizer.step()

        total_reg_loss += reg_loss.item() * source_data.size(0)
        total_samples += source_data.size(0)
        total_domain_loss += domain_loss.item() * (len(source_data) + len(target_data))

    avg_reg_loss = total_reg_loss / total_samples
    avg_domain_loss = total_domain_loss / (total_samples + len(target_loader.dataset))

    return avg_reg_loss, avg_domain_loss

In [ ]:
def domain_valid(source_valid_loader, model, loss_fn, epoch, show=False):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for data, label in enumerate(source_valid_loader):
            data, label = data.to(device), label.to(device)

            output = model(data)

            loss = loss_fn(output, label)

            total_loss += loss.item() * data.size(0)
            total_samples += data.size(0)

    avg_loss = total_loss / total_samples

    return avg_loss

In [ ]:
num_epoch = 70
#70

history = {'train_loss':[], 'val_loss':[], 'train_domain':[]}
early_stopping = EarlyStopping(patience = patience, verbose = True)
for t in range(num_epoch):
    train_loss, train_domain = domain_train(source_train_loader, target_loader, model,loss_fn, domain_loss_fn, t+1)
    val_loss = domain_valid(source_valid_loader, model, loss_fn, t+1)

    history['train_loss'].append(train_loss)
    history['train_domain'].append(train_domain)
    history['val_loss'].append(val_loss)

    print(f"#Training[{t+1}/{num_epoch}]: loss= {train_loss:>5f}, val_loss= {val_loss:>5f}, train_domain={train_domain:>5f}")
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

  0%|          | 0/31 [00:00<?, ?it/s]

#Training[1/70]: loss= 0.304589, val_loss= 0.258646, train_mae=0.226213, val_mae=0.188604, train_r2=-0.947136, val_r2=-0.325145


  0%|          | 0/31 [00:00<?, ?it/s]

#Training[2/70]: loss= 0.248496, val_loss= 0.236735, train_mae=0.182011, val_mae=0.176335, train_r2=-0.212686, val_r2=-0.042052


  0%|          | 0/31 [00:00<?, ?it/s]

...

#Training[68/70]: loss= 0.104219, val_loss= 0.119637, train_mae=0.094488, val_mae=0.095342, train_r2=0.585427, val_r2=0.617885


  0%|          | 0/31 [00:00<?, ?it/s]

#Training[69/70]: loss= 0.103683, val_loss= 0.119654, train_mae=0.093997, val_mae=0.095661, train_r2=0.578527, val_r2=0.586033
EarlyStopping counter: 1 out of 7


  0%|          | 0/31 [00:00<?, ?it/s]

#Training[70/70]: loss= 0.104122, val_loss= 0.119482, train_mae=0.094201, val_mae=0.095217, train_r2=0.578758, val_r2=0.622120
